<a href="https://colab.research.google.com/github/cthmayo/cough-diagosis/blob/main/COVID_Cough.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Introduction

This project used [cough data](https://www.kaggle.com/himanshu007121/coughclassifier-trial/code) in order to predict whether or not a patient had a diagnosis of COVID-19. A 1D CNN was used for [prediction](https://keras.io/examples/audio/speaker_recognition_using_cnn/).

# Imports

In [1]:
from google.colab import drive
from google.colab import files
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [2]:
!pip install tensorflow_io

     |████████████████████████████████| 22.7 MB 1.8 MB/s 
     |████████████████████████████████| 458.3 MB 9.3 kB/s 
     |████████████████████████████████| 462 kB 69.3 MB/s 
     |████████████████████████████████| 5.6 MB 65.6 MB/s 
     |████████████████████████████████| 1.3 MB 54.9 MB/s 
  Created wheel for clang: filename=clang-5.0-py3-none-any.whl size=30692 sha256=611bedddd3fc131abf8ffc4ae2fc8087f9d84ccfcb689fa9d4dc419360e15dd4
  Stored in directory: /root/.cache/pip/wheels/98/91/04/971b4c587cf47ae952b108949b46926f426c02832d120a082a
  Created wheel for wrapt: filename=wrapt-1.12.1-cp37-cp37m-linux_x86_64.whl size=68721 sha256=c2ef7455ccc75ed4d267fb1dd6971add464778a2bc06f171f0622ea54548666e
  Stored in directory: /root/.cache/pip/wheels/62/76/4c/aa25851149f3f6d9785f6c869387ad82b3fd37582fa8147ac6
Successfully built clang wrapt
  Attempting uninstall: typing-extensions
    Found existing installation: typing-extensions 3.10.0.2
    Uninstalling typing-extensions-3.10.0.2:
      Succe

In [3]:
import pandas as pd
import numpy as np
from tqdm import tqdm
from tqdm.notebook import tqdm_notebook
tqdm_notebook.pandas()
from zipfile import ZipFile
from io import StringIO
import matplotlib.pyplot as plt
np.random.seed(42)
import tensorflow as tf
from tensorflow import keras
import tensorflow_io as tfio
from IPython.display import Audio

In [4]:
z = ZipFile('/content/drive/MyDrive/COVID Cough Project/archive.zip')
z.extractall()

In [5]:
SAMPLING_RATE = 16000
FILE_PATH = 'trial_covid/'

In [6]:
df_class = pd.read_csv('cough_trial_extended.csv')

In [7]:
df_class['label'] = df_class['class'].apply(lambda x: {'not_covid':0, 'covid':1}[x])

# EDA

In [8]:
df_class

,file_properties,class,label
0,0v8MGxNetjg_ 10.000_ 20.000.wav,not_covid,0
1,1j1duoxdxBg_ 70.000_ 80.000.wav,not_covid,0
2,1MSYO4wgiag_ 120.000_ 130.000.wav,not_covid,0
3,1PajbAKd8Kg_ 0.000_ 10.000.wav,not_covid,0
4,cov1.wav,covid,1
...,...,...,...
165,-bZrDCS8KAg_ 70.000_ 80.000.wav,not_covid,0
166,-ej81N6Aqo4_ 0.000_ 8.000.wav,not_covid,0
167,-gvLnl1smfs_ 90.000_ 100.000.wav,not_covid,0
168,-hu5q-Nn4BM_ 70.000_ 80.000.wav,not_covid,0


In [9]:
df_class['label'].value_counts()

0    151
1     19
Name: label, dtype: int64

# Audio processing

Resample audio to 22050Hz:

In [10]:
STANDARD_RATE = 22050
VALID_SPLIT = 0.1
BATCH_SIZE = 8
FFT_SHAPE = 16384

In [11]:
def play_audio(sample):
  try:
    return Audio(sample.numpy().T,rate=STANDARD_RATE)
  except:
    return Audio(sample.T,rate=STANDARD_RATE)

In [12]:
def path_to_audio(path):
    sample, sampling_rate = tf.audio.decode_wav(tf.io.read_file(FILE_PATH+path), desired_channels=1)
    sampling_rate = tf.cast(sampling_rate, tf.int64)
    if sampling_rate != STANDARD_RATE:
      resampled_sample = tfio.audio.resample(sample, sampling_rate, STANDARD_RATE)
    else:
      resampled_sample = sample
    return resampled_sample

Play some audio

In [13]:
play_audio(path_to_audio('cov1.wav'))

In [14]:
def paths_and_labels_to_dataset(audio_paths, labels):
  path_ds = tf.data.Dataset.from_tensor_slices(audio_paths)
  audio_ds = path_ds.map(lambda x: path_to_audio(x))
  label_ds = tf.data.Dataset.from_tensor_slices(labels)
  return tf.data.Dataset.zip((audio_ds, label_ds))

In [15]:
@tf.function
def trim_zeros(t):
  return tf.expand_dims(tf.RaggedTensor.from_tensor(t, padding=0).merge_dims(0,1), axis=-1)

In [16]:
def audio_to_fft(audio):
  audio = trim_zeros(audio)
  audio = tf.squeeze(audio, axis=-1)
  fft = tf.signal.rfft(audio, tf.constant(FFT_SHAPE, shape=(1,)))
  fft = tf.expand_dims(fft, axis=-1)
  return tf.math.abs(fft)

In [17]:
df_class = df_class.sample(frac=1, random_state=42).reset_index()

In [18]:
num_val_samples = int(VALID_SPLIT * len(df_class))

Create training and validation datasets

In [19]:
df_train = df_class[:-num_val_samples]
df_val = df_class[-num_val_samples:]

In [20]:
train_ds = paths_and_labels_to_dataset(df_train['file_properties'].values, df_train['label'].values)
val_ds = paths_and_labels_to_dataset(df_val['file_properties'].values, df_val['label'].values)

Do a FFT on the audio, and batch for training...

In [21]:
train_ds = train_ds.map(
    lambda x, y: (audio_to_fft(x), y), num_parallel_calls=tf.data.AUTOTUNE
)
train_ds = train_ds.prefetch(tf.data.AUTOTUNE)

In [22]:
val_ds = val_ds.map(
    lambda x, y: (audio_to_fft(x), y), num_parallel_calls=tf.data.AUTOTUNE
)
val_ds = val_ds.prefetch(tf.data.AUTOTUNE)

In [23]:
train_ds = train_ds.shuffle(buffer_size=BATCH_SIZE * 8, seed=42).batch(BATCH_SIZE)
val_ds = val_ds.shuffle(buffer_size=BATCH_SIZE * 8, seed=42).batch(BATCH_SIZE)

# Model

Model is a 1D CNN from [here](https://keras.io/examples/audio/speaker_recognition_using_cnn/).

In [24]:
def residual_block(x, filters, conv_num=3, activation="relu"):
    # Shortcut
    s = keras.layers.Conv1D(filters, 1, padding="same")(x)
    for i in range(conv_num - 1):
        x = keras.layers.Conv1D(filters, 3, padding="same")(x)
        x = keras.layers.Activation(activation)(x)
    x = keras.layers.Conv1D(filters, 3, padding="same")(x)
    x = keras.layers.Add()([x, s])
    x = keras.layers.Activation(activation)(x)
    return keras.layers.MaxPool1D(pool_size=2, strides=2)(x)


def build_model(input_shape, num_classes):
    inputs = keras.layers.Input(shape=input_shape, name="input")

    x = residual_block(inputs, 16, 2)
    x = residual_block(x, 32, 2)
    x = residual_block(x, 64, 3)
    x = residual_block(x, 128, 3)
    x = residual_block(x, 128, 3)

    x = keras.layers.AveragePooling1D(pool_size=3, strides=3)(x)
    x = keras.layers.Flatten()(x)
    x = keras.layers.Dense(256, activation="relu")(x)
    x = keras.layers.Dense(128, activation="relu")(x)

    outputs = keras.layers.Dense(num_classes, activation="sigmoid", name="output")(x)

    return keras.models.Model(inputs=inputs, outputs=outputs)

In [25]:
model = build_model((FFT_SHAPE // 2 + 1, 1), 1)

In [26]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input (InputLayer)              [(None, 8193, 1)]    0                                            
__________________________________________________________________________________________________
conv1d_1 (Conv1D)               (None, 8193, 16)     64          input[0][0]                      
__________________________________________________________________________________________________
activation (Activation)         (None, 8193, 16)     0           conv1d_1[0][0]                   
__________________________________________________________________________________________________
conv1d_2 (Conv1D)               (None, 8193, 16)     784         activation[0][0]                 
______________________________________________________________________________________________

In [27]:
model.compile(
    optimizer="Adam", loss="binary_crossentropy", metrics=["accuracy"]
)

In [28]:
model_save_filename = "model.h5"

earlystopping_cb = keras.callbacks.EarlyStopping(patience=5, restore_best_weights=True)
mdlcheckpoint_cb = keras.callbacks.ModelCheckpoint(
    model_save_filename, monitor="val_accuracy", save_best_only=True
)

In [29]:
EPOCHS = 100

In [30]:
history = model.fit(
    train_ds,
    epochs=EPOCHS,
    validation_data=val_ds,
    callbacks=[earlystopping_cb, mdlcheckpoint_cb],
)

Epoch 1/100
20/20 [==============================] - 17s 624ms/step - loss: 0.5198 - accuracy: 0.8824 - val_loss: 0.1592 - val_accuracy: 0.8824


/usr/local/lib/python3.7/dist-packages/keras/utils/generic_utils.py:497: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  category=CustomMaskWarning)


Epoch 2/100
20/20 [==============================] - 14s 600ms/step - loss: 0.3484 - accuracy: 0.8889 - val_loss: 0.1779 - val_accuracy: 0.8824
Epoch 3/100
20/20 [==============================] - 14s 611ms/step - loss: 0.3684 - accuracy: 0.8889 - val_loss: 0.2290 - val_accuracy: 0.8824
Epoch 4/100
20/20 [==============================] - 14s 606ms/step - loss: 0.2450 - accuracy: 0.8889 - val_loss: 0.1523 - val_accuracy: 0.8824
Epoch 5/100
20/20 [==============================] - 15s 616ms/step - loss: 0.2272 - accuracy: 0.9085 - val_loss: 0.1325 - val_accuracy: 0.8824
Epoch 6/100
20/20 [==============================] - 15s 614ms/step - loss: 0.1824 - accuracy: 0.8889 - val_loss: 0.1305 - val_accuracy: 0.8824
Epoch 7/100
20/20 [==============================] - 15s 620ms/step - loss: 0.1617 - accuracy: 0.9216 - val_loss: 0.0958 - val_accuracy: 0.8824
Epoch 8/100
20/20 [==============================] - 15s 623ms/step - loss: 0.2388 - accuracy: 0.8889 - val_loss: 0.1140 - val_accuracy:

A 1D CNN is able to quite quickly reach 100% validation accuracy on a small (170 sample) dataset.